In [ ]:
"""
# -----------------------------------------------------------------------------
# PURPOSE: NIAP TO PRODUCT MAPPING
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
"""

import pandas as pd
import json
import re
import time
import numpy as np
import traceback
from datetime import datetime
from rapidfuzz import process, fuzz, utils

# 1. Configuration
TARGET_FOLDER_ID = "15xKraT4MhWzBvOjMTJ0x21TBIopqfl4E"
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")

# Artifacts
CSV_JIRA = f"raw_jira_initiatives_{TIMESTAMP}.csv"
CSV_REG  = f"raw_niap_register_{TIMESTAMP}.csv"
CSV_CORE = f"raw_core_inventory_{TIMESTAMP}.csv"
CSV_MASTER = f"master_reconciliation_table_{TIMESTAMP}.csv"
JSON_METRICS = f"data_overview_metrics_{TIMESTAMP}.json"
JSON_ANALYSIS = f"mapping_metrics_analysis_{TIMESTAMP}.json"
REPORT_MD = f"audit_analysis_report_{TIMESTAMP}.md"
TXT_METADATA = f"audit_provenance_metadata_{TIMESTAMP}.txt"

# -----------------------------------------------------------------------------
# UTILITIES
# -----------------------------------------------------------------------------
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        if isinstance(obj, np.floating): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super(NpEncoder, self).default(obj)

def clean_technical_name(text):
    if pd.isna(text): return ""
    text = str(text).upper()
    text = re.sub(r'_(GB|EUR|US|AU|NZ|SG|JP|RO|FR|PL|ES|IT|IE)_?', ' ', text)
    identifiers = [r'PT_', r'MF_', r'ACQ_', r'REVX_', r'_REPRICING', r'_PLAN', r'_BASE', r'_STD', r'_PREM', r'_METAL']
    for pattern in identifiers: 
        text = re.sub(pattern, ' ', text)
    text = re.sub(r'[^A-Z0-9\s]', ' ', text) 
    return " ".join(text.split()).lower()

class SmartMatcher:
    def __init__(self, source_list):
        self.original_source = [str(x) for x in source_list if pd.notna(x)]
        self.clean_source = [utils.default_process(x) for x in self.original_source]

    def match(self, query_list, threshold=82):
        results = []
        for query in query_list:
            if not query or str(query).strip() == "":
                results.append({"best_match": None, "score": 0})
                continue
            match = process.extractOne(utils.default_process(str(query)), self.clean_source, scorer=fuzz.WRatio, processor=None)
            if match and match[1] >= threshold:
                results.append({"best_match": self.original_source[match[2]], "score": round(match[1], 2)})
            else:
                results.append({"best_match": None, "score": 0})
        return results

# -----------------------------------------------------------------------------
# REPORT GENERATORS
# -----------------------------------------------------------------------------
def generate_audit_manifest(exec_time, sql_map):
    content = f"""NIAP AUDIT: FORENSIC METADATA MANIFEST
================================================================================
EXECUTION TIMESTAMP: {exec_time}
ALGORITHM: RapidFuzz (WRatio) | THRESHOLD: 82%
NORMALIZATION REGEX: r'_(GB|EUR|US...)', r'PT_|MF_|ACQ_|REVX_', r'_REPRICING|_PLAN...'

DATA PROVENANCE (EXACT SQL EXECUTION):
--------------------------------------------------------------------------------
1. JIRA SOURCE:
   {sql_map['jira']}

2. REGISTER SOURCE:
   {sql_map['reg']}

3. CORE INVENTORY SOURCE:
   {sql_map['core']}

LOGIC HIERARCHY:
--------------------------------------------------------------------------------
1. B1 (Reg->Jira): Match Register Name to Jira Summary. Priority: Direct NIAP ID > Fuzzy Match.
2. B2 (Core->Reg): Match Cleaned Core Name to Register Name. If found, inherit Jira Ticket.
3. B3 (Core->Jira): Match Cleaned Core Name to Jira Summary. Only runs if B2 fails.
"""
    with open(TXT_METADATA, 'w') as f: f.write(content)

def generate_deep_dive_report(metrics_a, metrics_b):
    # 1. Jira Status Table
    jira_table = "| Status | Total Tickets | Linked to Reg | Coverage % |\n|---|---|---|---|\n"
    for status, data in metrics_a['a1_jira_governance']['linkage_analysis'].items():
        jira_table += f"| {status} | {data['total_tickets']} | {data['tickets_with_link']} | {data['coverage_pct']}% |\n"

    # 2. Register Health Table
    reg_table = "| Product Status | Total Records | Populated NIAP IDs |\n|---|---|---|\n"
    for status, data in metrics_a['a2_register_health']['id_health_by_status'].items():
        reg_table += f"| {status} | {data['total_records']} | {data['populated_ids']} |\n"

    # 3. Core Inventory Breakdown
    core_table = "| Product Type | Active Configurations |\n|---|---|\n"
    sorted_core = dict(sorted(metrics_a['a3_core_summary']['breakdown_by_type'].items(), key=lambda item: item[1], reverse=True))
    for p_type, count in sorted_core.items():
        core_table += f"| {p_type} | {count} |\n"

    # 4. Gap Analysis Table
    gap_table = "| Product Type | Risk Volume (Unmapped) |\n|---|---|\n"
    gap_data = metrics_b['master_summary']['gap_risk_volume_by_type']
    for p_type, vol in gap_data.items():
        gap_table += f"| {p_type} | {vol} |\n"

    md = f"""# NIAP Audit: Deep Dive Analysis Report
**Timestamp:** {datetime.now().strftime('%Y-%m-%d %H:%M')}

## 1. Executive Summary
| Metric | Value |
|---|---|
| **Total Product Families** | {metrics_b['master_summary']['total_families']} |
| **Compliant Families** | {metrics_b['master_summary']['compliant']} |
| **Gap Families** | {metrics_b['master_summary']['gaps']} |
| **Total Risk Volume** | {metrics_a['a3_core_summary']['total_configs']} |

## 2. Phase A: Source Data Health

### 2.1 Jira Governance Linkage (By Status)
{jira_table}

### 2.2 Product Registry Health (By Status)
{reg_table}

### 2.3 Core Inventory Composition (Risk Profile)
{core_table}

## 3. Phase B: Mapping & Gap Analysis

### 3.1 Unmapped Risk Volumes (Gaps)
{gap_table}
"""
    with open(REPORT_MD, 'w') as f: f.write(md)

# -----------------------------------------------------------------------------
# ROBUST UPLOAD WRAPPER
# -----------------------------------------------------------------------------
def upload_to_drive(file_path, mime_type):
    max_retries = 3
    print(f" -> Uploading {file_path}...")
    
    for attempt in range(max_retries):
        try:
            upload_file_to_gdrive(file=file_path, name=file_path, mime_type=mime_type, folder_id=TARGET_FOLDER_ID)
            return # Success
        except NameError:
            print(f"    [WARN] Upload skipped (Local Mode).")
            return
        except Exception as e:
            print(f"    [WARN] Attempt {attempt+1}/{max_retries} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(5) # Wait 5 seconds before retrying
            else:
                print(f"    [ERROR] Failed to upload {file_path} after {max_retries} attempts.")

# -----------------------------------------------------------------------------
# MAIN EXECUTION
# -----------------------------------------------------------------------------
def run_master_audit():
    try:
        print(f"[{datetime.now()}] Phase A: Extraction...")
        
        queries = {
            "jira": "SELECT issue_id, summary, issue_status, product_register_link FROM global_entity_operations.jira_niap_initiatives",
            "reg": "SELECT name, niap, product_status FROM global_entity_operations.niap_product_register",
            "core": "SELECT product_type, name, COUNT(*) as configuration_count FROM core.products WHERE (decommission_date IS NULL OR decommission_date > CURRENT_DATE) GROUP BY 1, 2"
        }

        with zeus() as cur:
            df_jira = execute_sql(cur, queries["jira"], None)
            df_reg  = execute_sql(cur, queries["reg"], None)
            df_core = execute_sql(cur, queries["core"], None)

        # --- METRICS A ---
        df_jira['has_reg_link'] = df_jira['product_register_link'].notna() & (df_jira['product_register_link'].astype(str) != 'None')
        jira_stats = df_jira.groupby('issue_status').agg(total_tickets=('issue_id', 'count'), tickets_with_link=('has_reg_link', 'sum'))
        jira_stats['coverage_pct'] = (jira_stats['tickets_with_link'] / jira_stats['total_tickets'] * 100).round(2)

        df_reg['niap_populated'] = df_reg['niap'].notna() & (~df_reg['niap'].astype(str).isin(['None', 'nan', '']))
        df_reg['product_status'] = df_reg['product_status'].fillna('Unknown')
        reg_health = df_reg.groupby('product_status').agg(total_records=('name', 'count'), populated_ids=('niap_populated', 'sum'))

        metrics_a = {
            "a1_jira_governance": {"total_initiatives": len(df_jira), "linkage_analysis": jira_stats.to_dict('index')},
            "a2_register_health": {"total": len(df_reg), "id_health_by_status": reg_health.to_dict('index')},
            "a3_core_summary": {
                "total_configs": int(df_core['configuration_count'].sum()),
                "breakdown_by_type": df_core.groupby('product_type')['configuration_count'].sum().to_dict()
            }
        }

        # --- PHASE B1 (Reg -> Jira) ---
        print(f"[{datetime.now()}] Phase B1: Register Mapping...")
        j_matcher = SmartMatcher(df_jira['summary'].tolist())
        j_id_map = dict(zip(df_jira['summary'], df_jira['issue_id']))
        j_status_map = dict(zip(df_jira['issue_id'], df_jira['issue_status']))
        
        b1_results = {}
        b1_matches = j_matcher.match(df_reg['name'].tolist())
        
        for i, row in df_reg.iterrows():
            name = str(row['name'])
            key = str(row['niap']) if pd.notna(row['niap']) else "nan"
            match = b1_matches[i]
            if key in j_status_map: res = {"ticket": key, "method": "Direct Link", "conf": 100}
            elif match['best_match']: res = {"ticket": j_id_map.get(match['best_match']), "method": "Fuzzy (WRatio)", "conf": match['score']}
            else: res = {"ticket": "-", "method": "GAP", "conf": 0}
            b1_results[name.lower()] = res

        # --- PHASE B2/B3 (Core -> Reg -> Jira) ---
        print(f"[{datetime.now()}] Phase B2/B3: Hierarchical Mapping...")
        core_families = df_core.groupby(['product_type', 'name'])['configuration_count'].sum().reset_index()
        core_families['clean_name'] = core_families['name'].apply(clean_technical_name)
        
        r_matcher = SmartMatcher(df_reg['name'].tolist())
        b2_matches = r_matcher.match(core_families['clean_name'].tolist(), threshold=85)
        b3_matches = j_matcher.match(core_families['clean_name'].tolist(), threshold=85)

        master_results = []
        for i, c_row in core_families.iterrows():
            ticket, path, best_reg = "-", "GAP", "-"
            
            if b2_matches[i]['best_match']:
                best_reg = b2_matches[i]['best_match']
                link = b1_results.get(best_reg.lower())
                if link and link['ticket'] != "-": ticket, path = link['ticket'], f"Inherited via Register ({link['method']})"
            
            if ticket == "-" and b3_matches[i]['best_match']:
                 found_id = j_id_map.get(b3_matches[i]['best_match'])
                 if found_id: ticket, path = found_id, "Direct Match to Jira"

            master_results.append({
                "product_family": c_row['name'], "product_type": c_row['product_type'], "risk_volume": int(c_row['configuration_count']), 
                "mapped_reg_entry": best_reg, "evidence_ticket": ticket, "status": j_status_map.get(ticket, "N/A"), "traceability_path": path
            })

        # --- FINAL OUTPUTS ---
        print(f"[{datetime.now()}] Generating Deep Dive Reports...")
        df_master = pd.DataFrame(master_results)
        df_master.to_csv(CSV_MASTER, index=False)
        
        gap_data = df_master[df_master['evidence_ticket'] == "-"]
        metrics_b = {
            "b1_registry_health": b1_results,
            "master_summary": {
                "total_families": len(df_master), "compliant": len(df_master) - len(gap_data), "gaps": len(gap_data),
                "gap_risk_volume_by_type": gap_data.groupby('product_type')['risk_volume'].sum().sort_values(ascending=False).to_dict()
            }
        }
        
        with open(JSON_METRICS, 'w') as f: json.dump(metrics_a, f, indent=4, cls=NpEncoder)
        with open(JSON_ANALYSIS, 'w') as f: json.dump(metrics_b, f, indent=4, cls=NpEncoder)
        generate_audit_manifest(str(datetime.now()), queries)
        generate_deep_dive_report(metrics_a, metrics_b)
        
        df_jira.to_csv(CSV_JIRA, index=False) 
        df_reg.to_csv(CSV_REG, index=False)
        df_core.to_csv(CSV_CORE, index=False)

        print(f"[{datetime.now()}] Uploading 8 Artifacts...")
        artifacts = [
            (CSV_MASTER, "text/csv"), (JSON_METRICS, "application/json"), 
            (JSON_ANALYSIS, "application/json"), (TXT_METADATA, "text/plain"), 
            (REPORT_MD, "text/markdown"), (CSV_JIRA, "text/csv"), 
            (CSV_REG, "text/csv"), (CSV_CORE, "text/csv")
        ]
        for f_path, f_mime in artifacts:
            upload_to_drive(f_path, f_mime)

        print(f"\n✅ DEEP DIVE AUDIT COMPLETE. Report: {REPORT_MD}")

    except Exception as e:
        print(f"\n❌ EXECUTION FAILED: {str(e)}")
        traceback.print_exc()

if __name__ == "__main__":
    run_master_audit()